In [1]:
from langchain.chat_models import init_chat_model
chat_model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [2]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEndpoint

In [3]:
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [16]:
repo_id = "EleutherAI/gpt-j-6b"
import os
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    temperature=0.5,
)
llm_chain = prompt | chat_model

# Few Shots prompting

In [6]:
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
system_prompt = SystemMessagePromptTemplate.from_template(
 """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Context: {context}
"""
)

In [7]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        HumanMessagePromptTemplate.from_template("{query}"),
    ]
)

In [8]:
pipeline = (
    {"query": lambda x : x["query"],
    "context": lambda x : x["context"]
    }
    | prompt
    | chat_model
)

In [9]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [10]:

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}"),
])

Then we define a list of examples with dictionaries containing the correct `input` and `output` keys.

In [11]:
examples = [
    {"input": "Here is query #1", "output": "Here is the answer #1"},
    {"input": "Here is query #2", "output": "Here is the answer #2"},
    {"input": "Here is query #3", "output": "Here is the answer #3"},
]

We then feed both of these into our `FewShotChatMessagePromptTemplate` object:

In [12]:
from langchain.prompts import FewShotChatMessagePromptTemplate

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: Here is query #1
AI: Here is the answer #1
Human: Here is query #2
AI: Here is the answer #2
Human: Here is query #3
AI: Here is the answer #3


In [13]:
context = """Aurelio AI is an AI company developing tooling for AI
engineers. Their focus is on language AI with the team having strong
expertise in building AI agents and a strong background in
information retrieval.

The company is behind several open source frameworks, most notably
Semantic Router and Semantic Chunkers. They also have an AI
Platform providing engineers with tooling to help them build with
AI. Finally, the team also provides development services to other
organizations to help them bring their AI tech to market.

Aurelio AI became LangChain Experts in September 2024 after a long
track record of delivering AI solutions built with the LangChain
ecosystem."""

query = "what does Aurelio AI do?"

In [17]:
new_system_prompt = """
Answer the user's query based on the context below.
If you cannot answer the question using the
provided information answer with "I don't know".

Always answer in markdown format. When doing so please
provide headers, short summaries, follow with bullet
points, then conclude.

Context: {context}
"""

prompt.messages[0].prompt.template = new_system_prompt

out = pipeline.invoke({"query": query, "context": context})
print(out)

content='# Aurelio AI Overview\n\nAurelio AI is an AI company that specializes in tooling for AI engineers, with a focus on language AI.\n\n*   **Expertise:** They have strong expertise in building AI agents and a strong background in information retrieval.\n*   **Open Source Frameworks:** They are behind several open-source frameworks, including Semantic Router and Semantic Chunkers.\n*   **AI Platform:** They offer an AI Platform that provides engineers with tools to help them build with AI.\n*   **Development Services:** They provide development services to help other organizations bring their AI tech to market.\n*   **LangChain Experts:** They became LangChain Experts in September 2024.\n\nIn summary, Aurelio AI offers a range of services and tools centered around language AI, including open-source frameworks, an AI platform, and development services, leveraging their expertise in AI agents, information retrieval, and the LangChain ecosystem.' additional_kwargs={} response_metadata

In [18]:
from IPython.display import display, Markdown

display(Markdown(out.content))

# Aurelio AI Overview

Aurelio AI is an AI company that specializes in tooling for AI engineers, with a focus on language AI.

*   **Expertise:** They have strong expertise in building AI agents and a strong background in information retrieval.
*   **Open Source Frameworks:** They are behind several open-source frameworks, including Semantic Router and Semantic Chunkers.
*   **AI Platform:** They offer an AI Platform that provides engineers with tools to help them build with AI.
*   **Development Services:** They provide development services to help other organizations bring their AI tech to market.
*   **LangChain Experts:** They became LangChain Experts in September 2024.

In summary, Aurelio AI offers a range of services and tools centered around language AI, including open-source frameworks, an AI platform, and development services, leveraging their expertise in AI agents, information retrieval, and the LangChain ecosystem.

In [55]:
examples = [
    {
        "input": "Can you explain gravity?",
        "output": (
            "## Gravity\n\n"
            "Gravity is one of the fundamental forces in the universe.\n\n"
            "### Discovery\n\n"
            "* Gravity was first discovered by Sir Isaac Newton in the late 17th century.\n"
            "* It was said that Newton theorized about gravity after seeing an apple fall from a tree.\n\n"
            "### In General Relativity\n\n"
            "* Gravity is described as the curvature of spacetime.\n"
            "* The more massive an object is, the more it curves spacetime.\n"
            "* This curvature is what causes objects to fall towards each other.\n\n"
            "### Gravitons\n\n"
            "* Gravitons are hypothetical particles that mediate the force of gravity.\n"
            "* They have not yet been detected.\n\n"
            "**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.\n\n"
        )
    },
    {
        "input": "What is the capital of France?",
        "output": (
            "## France\n\n"
            "The capital of France is Paris.\n\n"
            "### Origins\n\n"
            "* The name Paris comes from the Latin word \"Parisini\" which referred to a Celtic people living in the area.\n"
            "* The Romans named the city Lutetia, which means \"the place where the river turns\".\n"
            "* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.\n\n"
            "**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.\n\n"
        )
    }
]

In [56]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [57]:
out = few_shot_prompt.format()

display(Markdown(out))

Human: Can you explain gravity?
AI: ## Gravity

Gravity is one of the fundamental forces in the universe.

### Discovery

* Gravity was first discovered by Sir Isaac Newton in the late 17th century.
* It was said that Newton theorized about gravity after seeing an apple fall from a tree.

### In General Relativity

* Gravity is described as the curvature of spacetime.
* The more massive an object is, the more it curves spacetime.
* This curvature is what causes objects to fall towards each other.

### Gravitons

* Gravitons are hypothetical particles that mediate the force of gravity.
* They have not yet been detected.

**To conclude**, Gravity is a fascinating topic and has been studied extensively since the time of Newton.


Human: What is the capital of France?
AI: ## France

The capital of France is Paris.

### Origins

* The name Paris comes from the Latin word "Parisini" which referred to a Celtic people living in the area.
* The Romans named the city Lutetia, which means "the place where the river turns".
* The city was renamed Paris in the 3rd century BC by the Celtic-speaking Parisii tribe.

**To conclude**, Paris is highly regarded as one of the most beautiful cities in the world and is one of the world's greatest cultural and economic centres.



In [58]:
prompt_template = ChatPromptTemplate.from_messages([
    ("system", new_system_prompt),
    few_shot_prompt,
    ("user", "{query}"),
])

In [60]:
pipeline = prompt_template | chat_model
out = pipeline.invoke({"query": query, "context": context})
display(Markdown(out.content))

## Aurelio AI

Aurelio AI is an AI company that focuses on providing tools and services for AI engineers, with a specialization in language AI.

### Summary

*   **Focus:** Language AI
*   **Expertise:** Building AI agents and information retrieval
*   **Open Source Frameworks:** Semantic Router and Semantic Chunkers
*   **AI Platform:** Provides tooling for AI engineers.
*   **Development Services:** Helps organizations bring their AI tech to market.
*   **LangChain Experts:** Became LangChain Experts in September 2024.

## Chain of Thought Prompting

In [61]:
no_cot_system_prompt = """
Be a helpful assistant and answer the user's question.

You MUST answer the question directly without any other
text or explanation.
"""

no_cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", no_cot_system_prompt),
    ("user", "{query}"),
])

In [64]:
query = (
    "How many keystrokes are needed to type the numbers from 1 to 500?"
)

no_cot_pipeline = no_cot_prompt_template | chat_model
no_cot_result = no_cot_pipeline.invoke({"query": query})
print(no_cot_result.content)

1401


The actual answer is `1392`, but the LLM _without_ CoT just hallucinates and gives us a guess. Now, we can add explicit CoT prompting to our system prompt to see if we can get a better result.

In [65]:
# Define the chain-of-thought prompt template
cot_system_prompt = """
Be a helpful assistant and answer the user's question.

To answer the question, you must:

- List systematically and in precise detail all
  subproblems that need to be solved to answer the
  question.
- Solve each sub problem INDIVIDUALLY and in sequence.
- Finally, use everything you have worked through to
  provide the final answer.
"""

cot_prompt_template = ChatPromptTemplate.from_messages([
    ("system", cot_system_prompt),
    ("user", "{query}"),
])

cot_pipeline = cot_prompt_template | chat_model

In [66]:
cot_result = cot_pipeline.invoke({"query": query}).content
display(Markdown(cot_result))

Here's how to solve this problem step-by-step:

**1. Identify the Subproblems**

We need to break this down by the number of digits in each number:

*   **Subproblem 1:** Count the number of 1-digit numbers (1-9) and the keystrokes needed to type them.
*   **Subproblem 2:** Count the number of 2-digit numbers (10-99) and the keystrokes needed to type them.
*   **Subproblem 3:** Count the number of 3-digit numbers (100-500) and the keystrokes needed to type them.
*   **Subproblem 4:** Calculate the total keystrokes.

**2. Solve Subproblem 1: 1-digit numbers**

*   Numbers: 1, 2, 3, 4, 5, 6, 7, 8, 9
*   Count: 9 numbers
*   Keystrokes per number: 1
*   Total keystrokes: 9 * 1 = 9

**3. Solve Subproblem 2: 2-digit numbers**

*   Numbers: 10 to 99
*   Count: 99 - 10 + 1 = 90 numbers
*   Keystrokes per number: 2
*   Total keystrokes: 90 * 2 = 180

**4. Solve Subproblem 3: 3-digit numbers**

*   Numbers: 100 to 500
*   Count: 500 - 100 + 1 = 401 numbers
*   Keystrokes per number: 3
*   Total keystrokes: 401 * 3 = 1203

**5. Solve Subproblem 4: Calculate the total keystrokes**

*   Total keystrokes = Keystrokes (1-digit) + Keystrokes (2-digit) + Keystrokes (3-digit)
*   Total keystrokes = 9 + 180 + 1203 = 1392

**Final Answer:** It takes 1392 keystrokes to type the numbers from 1 to 500.

Now let's try without specifiying the use of cot 

In [67]:
system_prompt = """
Be a helpful assistant and answer the user's question.
"""

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("user", "{query}"),
])

pipeline = prompt_template | chat_model

In [68]:
result = pipeline.invoke({"query": query}).content
display(Markdown(result))

Okay, let's break this down:

*   **1-Digit Numbers (1-9):** There are 9 numbers, each requiring 1 keystroke. Total: 9 * 1 = 9 keystrokes.

*   **2-Digit Numbers (10-99):** There are 90 numbers, each requiring 2 keystrokes. Total: 90 * 2 = 180 keystrokes.

*   **3-Digit Numbers (100-500):** There are 401 numbers, each requiring 3 keystrokes. Total: 401 * 3 = 1203 keystrokes.

Now, add them up:

9 + 180 + 1203 = 1392

Therefore, it takes **1392** keystrokes to type the numbers from 1 to 500.